In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# import library
import os
import datetime
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import metrics, losses, optimizers
from tensorflow.keras import backend, layers, models, callbacks, utils
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications.xception import preprocess_input

from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix

from matplotlib import pyplot as plt

plt.style.use('dark_background')

In [3]:
# define plot function
def plot_loss(history):
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    epochs = range(1, len(loss) + 1)
    plt.plot(epochs, loss, label="training loss")
    plt.plot(epochs, val_loss, label="valid loss")
    plt.title("Training & Valid Loss")
    plt.legend()
    plt.show()


def plot_acc(history):
    acc = history.history["categorical_accuracy"]
    val_acc = history.history["val_categorical_accuracy"]
    epochs = range(1, len(acc) + 1)
    plt.plot(epochs, acc, label="training acc")
    plt.plot(epochs, val_acc, label="valid acc")
    plt.title("Training & Valid Accuracy")
    plt.legend()
    plt.show()

In [4]:
# define our working directory
dir_img_train = "dataset/subset/train/"
dir_img_validation = "dataset/subset/validation/"
dir_img_test = "dataset/subset/test/"
save_dir = os.path.join("models/")

In [5]:
img_size = (299, 299, 3)
batch_size = 16

lr_classifier = 1e-4
epoch_classifier = 10

lr_tuning = 1e-6
epoch_tuning = 12

In [6]:
train_datagen = image.ImageDataGenerator(
    # rescale=1. / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest")

# Note that the validation data should not be augmented!
test_datagen = image.ImageDataGenerator(
    # rescale=1. / 255
)

In [7]:
train_generator = train_datagen.flow_from_directory(
    # target folder yang berisi kumpulan citra
    dir_img_train,
    # semua citra diubah ukurannya menjadi 299 x 299
    target_size=img_size[:2],
    batch_size=batch_size,
    # class_mode categorical harus disesuaikan dengan fungsi loss
    # Karena fungsi loss yang digunakan adalah categorical crossentropy
    # maka class_mode = categorical
    class_mode="categorical",
    shuffle=True,
    seed=1234)

validation_generator = test_datagen.flow_from_directory(
    dir_img_validation,
    target_size=img_size[:2],
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    seed=1234)

test_generator = test_datagen.flow_from_directory(dir_img_test,
                                                  target_size=img_size[:2],
                                                  batch_size=batch_size,
                                                  class_mode="categorical",
                                                  shuffle=True,
                                                  seed=1234)

Found 12000 images belonging to 12 classes.
Found 1200 images belonging to 12 classes.
Found 1200 images belonging to 12 classes.


In [8]:
def create_model():
    conv_base = Xception(weights="imagenet",
                         include_top=False,
                         input_shape=img_size)

    conv_base.trainable = False

    input_layer = keras.Input(shape=img_size)
    x = preprocess_input(input_layer)
    x = conv_base(x, training=False)
    x1 = layers.GlobalAveragePooling2D()(x)
    x2 = layers.GlobalMaxPooling2D()(x)
    x = layers.Concatenate()([x1, x2])
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(512, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    output_layer = layers.Dense(12, activation="softmax")(x)

    model = keras.Model(inputs=input_layer, outputs=output_layer)

    return conv_base, model

In [9]:
strategy = tf.distribute.MirroredStrategy()
print("Number of devices: {}".format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of devices: 1


In [10]:
my_callbacks = [
    callbacks.CSVLogger(os.path.join(save_dir, "log_training.csv"),
                        separator=",",
                        append=True)
]

In [11]:
# Open a strategy scope.
with strategy.scope():
    # Everything that creates variables should be under the strategy scope.
    # In general this is only model construction & `compile()`.
    conv_base, model = create_model()
    model.compile(loss=losses.CategoricalCrossentropy(),
                  optimizer=optimizers.Adam(learning_rate=lr_classifier),
                  metrics=[metrics.CategoricalAccuracy()])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [12]:
conv_base.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
tf_op_layer_truediv (TensorFlow [(None, 299, 299, 3) 0           input_2[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_sub (TensorFlowOpLa [(None, 299, 299, 3) 0           tf_op_layer_truediv[0][0]        
__________________________________________________________________________________________________
xception (Model)                (None, 10, 10, 2048) 20861480    tf_op_layer_sub[0][0]            
______________________________________________________________________________________________

In [14]:
model.evaluate(test_generator, verbose=1)

  ...
    to  
  ['...']
75/75 [==============================] - 11s 147ms/step - loss: 2.9005 - categorical_accuracy: 0.1025


[2.900517609914144, 0.1025]

In [15]:
# memuat bobot model yang sudah dilatih bagian classifiernya sesuai dengan dataset ini
model.load_weights(
    os.path.join("transfer_learning_classifier.h5"),
    by_name=False,
    skip_mismatch=False)

In [16]:
model.evaluate(test_generator, verbose=1)

  ...
    to  
  ['...']
75/75 [==============================] - 10s 131ms/step - loss: 0.6193 - categorical_accuracy: 0.8083


[0.6193091050783793, 0.80833334]

# Fine tune from add 8

In [17]:
try:
    del (conv_base)
    del (model)
except:
    pass

In [18]:
backend.clear_session()

In [19]:
# Open a strategy scope.
with strategy.scope():
    # Everything that creates variables should be under the strategy scope.
    # In general this is only model construction & `compile()`.
    conv_base, model = create_model()
    model.compile(loss=losses.CategoricalCrossentropy(),
                  optimizer=optimizers.Adam(learning_rate=lr_classifier),
                  metrics=[metrics.CategoricalAccuracy()])

model.load_weights(
    os.path.join("transfer_learning_classifier.h5"),
    by_name=False,
    skip_mismatch=False)

In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
tf_op_layer_truediv (TensorFlow [(None, 299, 299, 3) 0           input_2[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_sub (TensorFlowOpLa [(None, 299, 299, 3) 0           tf_op_layer_truediv[0][0]        
__________________________________________________________________________________________________
xception (Model)                (None, 10, 10, 2048) 20861480    tf_op_layer_sub[0][0]            
______________________________________________________________________________________________

In [21]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'add_8':
        set_trainable = True

    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [22]:
with strategy.scope():
    model.compile(optimizer=optimizers.Adam(learning_rate=lr_tuning),
                  loss=losses.CategoricalCrossentropy(),
                  metrics=[metrics.CategoricalAccuracy()])

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
tf_op_layer_truediv (TensorFlow [(None, 299, 299, 3) 0           input_2[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_sub (TensorFlowOpLa [(None, 299, 299, 3) 0           tf_op_layer_truediv[0][0]        
__________________________________________________________________________________________________
xception (Model)                (None, 10, 10, 2048) 20861480    tf_op_layer_sub[0][0]            
______________________________________________________________________________________________

In [23]:
history = model.fit(train_generator,
                    epochs=epoch_tuning,
                    validation_data=validation_generator,
                    verbose=1,
                    callbacks=my_callbacks)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 750 steps, validate for 75 steps
Epoch 1/12
750/750 [==============================] - 173s 230ms/step - loss: 0.7450 - categorical_accuracy: 0.7517 - val_loss: 0.5820 - val_categorical_accuracy: 0.8083
Epoch 2/12
750/750 [==============================] - 164s 219ms/step - loss: 0.7422 - categorical_accuracy: 0.7523 - val_loss: 0.5834 - val_categorical_accuracy: 0.8050
Epoch 3/12
750/750 [==============================] - 164s 219ms/step - loss: 0.7274 - categorical_accuracy: 0.7574 - val_loss: 0.5786 - val_categorical_accuracy: 0.8058
Epoch 4/12
750/750 [==============================] - 165s 220ms/step - loss: 0.7256 - categorical_accuracy: 0.7581 - val_loss: 0.5787 - val_categorical_accuracy: 0.8083
Epoch 5/12
750/750 [==============================] - 164s 219ms/step - loss: 0.7130 - categorical_accuracy: 0.7613 - val_loss: 0.5780 - val_categorical_accuracy: 0.8058
Epoch 6/12
750/750 [==============================] - 164

## Evaluate Model

In [24]:
model.evaluate(test_generator, verbose=1)

  ...
    to  
  ['...']
75/75 [==============================] - 10s 133ms/step - loss: 0.5949 - categorical_accuracy: 0.8175


[0.5949116331338883, 0.8175]

# Fine tune from add 4

In [25]:
try:
    del (conv_base)
    del (model)
except:
    pass

In [26]:
backend.clear_session()

In [27]:
# Open a strategy scope.
with strategy.scope():
    # Everything that creates variables should be under the strategy scope.
    # In general this is only model construction & `compile()`.
    conv_base, model = create_model()
    model.compile(loss=losses.CategoricalCrossentropy(),
                  optimizer=optimizers.Adam(learning_rate=lr_classifier),
                  metrics=[metrics.CategoricalAccuracy()])

model.load_weights(
    os.path.join("transfer_learning_classifier.h5"),
    by_name=False,
    skip_mismatch=False)

In [28]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
tf_op_layer_truediv (TensorFlow [(None, 299, 299, 3) 0           input_2[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_sub (TensorFlowOpLa [(None, 299, 299, 3) 0           tf_op_layer_truediv[0][0]        
__________________________________________________________________________________________________
xception (Model)                (None, 10, 10, 2048) 20861480    tf_op_layer_sub[0][0]            
______________________________________________________________________________________________

In [29]:
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'add_4':
        set_trainable = True

    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [30]:
with strategy.scope():
    model.compile(optimizer=optimizers.Adam(learning_rate=lr_tuning),
                  loss=losses.CategoricalCrossentropy(),
                  metrics=[metrics.CategoricalAccuracy()])

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
tf_op_layer_truediv (TensorFlow [(None, 299, 299, 3) 0           input_2[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_sub (TensorFlowOpLa [(None, 299, 299, 3) 0           tf_op_layer_truediv[0][0]        
__________________________________________________________________________________________________
xception (Model)                (None, 10, 10, 2048) 20861480    tf_op_layer_sub[0][0]            
______________________________________________________________________________________________

In [31]:
history = model.fit(train_generator,
                    epochs=epoch_tuning,
                    validation_data=validation_generator,
                    verbose=1,
                    callbacks=my_callbacks)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 750 steps, validate for 75 steps
Epoch 1/12
750/750 [==============================] - 192s 256ms/step - loss: 0.7406 - categorical_accuracy: 0.7543 - val_loss: 0.5819 - val_categorical_accuracy: 0.8058
Epoch 2/12
750/750 [==============================] - 186s 248ms/step - loss: 0.7333 - categorical_accuracy: 0.7562 - val_loss: 0.5770 - val_categorical_accuracy: 0.8058
Epoch 3/12
750/750 [==============================] - 186s 248ms/step - loss: 0.7161 - categorical_accuracy: 0.7578 - val_loss: 0.5724 - val_categorical_accuracy: 0.8050
Epoch 4/12
750/750 [==============================] - 186s 248ms/step - loss: 0.7081 - categorical_accuracy: 0.7632 - val_loss: 0.5714 - val_categorical_accuracy: 0.8083
Epoch 5/12
750/750 [==============================] - 186s 248ms/step - loss: 0.6966 - categorical_accuracy: 0.7680 - val_loss: 0.5732 - val_categorical_accuracy: 0.8100
Epoch 6/12
750/750 [==============================] - 186

## Evaluate Model

In [32]:
model.evaluate(test_generator, verbose=1)

  ...
    to  
  ['...']
75/75 [==============================] - 10s 137ms/step - loss: 0.5950 - categorical_accuracy: 0.8167


[0.5950238787134489, 0.81666666]

# Fine tune all layer

In [33]:
try:
    del (conv_base)
    del (model)
except:
    pass

In [34]:
backend.clear_session()

In [35]:
# Open a strategy scope.
with strategy.scope():
    # Everything that creates variables should be under the strategy scope.
    # In general this is only model construction & `compile()`.
    conv_base, model = create_model()
    model.compile(loss=losses.CategoricalCrossentropy(),
                  optimizer=optimizers.Adam(learning_rate=lr_classifier),
                  metrics=[metrics.CategoricalAccuracy()])

model.load_weights(
    os.path.join("transfer_learning_classifier.h5"),
    by_name=False,
    skip_mismatch=False)

In [36]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
tf_op_layer_truediv (TensorFlow [(None, 299, 299, 3) 0           input_2[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_sub (TensorFlowOpLa [(None, 299, 299, 3) 0           tf_op_layer_truediv[0][0]        
__________________________________________________________________________________________________
xception (Model)                (None, 10, 10, 2048) 20861480    tf_op_layer_sub[0][0]            
______________________________________________________________________________________________

In [37]:
conv_base.trainable = True

In [38]:
with strategy.scope():
    model.compile(optimizer=optimizers.Adam(learning_rate=lr_tuning),
                  loss=losses.CategoricalCrossentropy(),
                  metrics=[metrics.CategoricalAccuracy()])

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
tf_op_layer_truediv (TensorFlow [(None, 299, 299, 3) 0           input_2[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_sub (TensorFlowOpLa [(None, 299, 299, 3) 0           tf_op_layer_truediv[0][0]        
__________________________________________________________________________________________________
xception (Model)                (None, 10, 10, 2048) 20861480    tf_op_layer_sub[0][0]            
______________________________________________________________________________________________

In [39]:
history = model.fit(train_generator,
                    epochs=epoch_tuning,
                    validation_data=validation_generator,
                    verbose=1,
                    callbacks=my_callbacks)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 750 steps, validate for 75 steps
Epoch 1/12
750/750 [==============================] - 437s 583ms/step - loss: 0.7297 - categorical_accuracy: 0.7552 - val_loss: 0.5749 - val_categorical_accuracy: 0.8142
Epoch 2/12
750/750 [==============================] - 431s 575ms/step - loss: 0.7139 - categorical_accuracy: 0.7601 - val_loss: 0.5704 - val_categorical_accuracy: 0.8158
Epoch 3/12
750/750 [==============================] - 432s 576ms/step - loss: 0.7096 - categorical_accuracy: 0.7599 - val_loss: 0.5679 - val_categorical_accuracy: 0.8150
Epoch 4/12
750/750 [==============================] - 432s 577ms/step - loss: 0.6929 - categorical_accuracy: 0.7669 - val_loss: 0.5637 - val_categorical_accuracy: 0.8158
Epoch 5/12
750/750 [==============================] - 432s 576ms/step - loss: 0.7007 - categorical_accuracy: 0.7672 - val_loss: 0.5599 - val_categorical_accuracy: 0.8208
Epoch 6/12
750/750 [==============================] - 431

## Evaluate Model

In [40]:
model.evaluate(test_generator, verbose=1)

  ...
    to  
  ['...']
75/75 [==============================] - 10s 133ms/step - loss: 0.5912 - categorical_accuracy: 0.8225


[0.5911683443188668, 0.8225]